<a href="https://colab.research.google.com/github/Abitha09062002/Seizure_Detection_Hyperparameter/blob/main/Project/Y_BayesianOptimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:



   from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
import pandas as pd
import numpy as np
import keras as ke
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection  import train_test_split
from sklearn.model_selection  import cross_val_score
from keras.models  import Sequential
from keras.layers import Dense, BatchNormalization, Dropout
from tensorflow.keras.optimizers import SGD, Adam, RMSprop, Adadelta, Adagrad, Adamax, Nadam, Ftrl
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.wrappers.scikit_learn import KerasClassifier
from math import floor
from sklearn.metrics import make_scorer, accuracy_score
#from bayes_opt import BayesianOptimization
import BayesianOptimization
#import bayes_opt.BayesianOptimization
from bayes_opt import BayesianOptimization
from sklearn.model_selection import StratifiedKFold
from keras.layers import LeakyReLU
LeakyReLU = LeakyReLU(alpha=0.1)
import warnings
warnings.filterwarnings('ignore')
pd.set_option("display.max_columns", None)


In [ ]:
d1=pd.read_csv('/content/gdrive/MyDrive/dataset-org.csv',header=None)
print(d1)


       0    1    2    3    4    5    6    7    8    9    10   11   12   13   \
0      135  190  229  223  192  125   55   -9  -33  -38  -10   35   64  113   
1      386  382  356  331  320  315  307  272  244  232  237  258  212    2   
2      -32  -39  -47  -37  -32  -36  -57  -73  -85  -94  -99  -94  -96 -104   
3     -105 -101  -96  -92  -89  -95 -102 -100  -87  -79  -72  -68  -74  -80   
4       -9  -65  -98 -102  -78  -48  -16    0  -21  -59  -90 -103  -84  -43   
...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
11495  -22  -22  -23  -26  -36  -42  -45  -42  -45  -49  -57  -64  -73  -79   
11496  -47  -11   28   77  141  211  246  240  193  136   78    8  -66 -132   
11497   14    6  -13  -16   10   26   27   -9    4   14   -1  -10   14   44   
11498  -40  -25   -9  -12   -2   12    7   19   22   29   22    6    1  -28   
11499   29   41   57   72   74   62   54   43   31   23   13   11   -3   -5   

       14   15   16    17    18    19   20   21   2

In [ ]:
score_acc = make_scorer(accuracy_score)

In [ ]:
X=pd.DataFrame(d1.iloc[:,:-1].values)
Y=d1.iloc[:,-1].values

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, random_state = 0 )

In [ ]:
def nn_cl_bo(neurons, activation, optimizer, learning_rate, batch_size, epochs ):
    optimizerL = ['SGD', 'Adam', 'RMSprop', 'Adadelta', 'Adagrad', 'Adamax', 'Nadam', 'Ftrl','SGD']
    optimizerD= {'Adam':Adam(lr=learning_rate), 'SGD':SGD(lr=learning_rate), 'RMSprop':RMSprop(lr=learning_rate), 'Adadelta':Adadelta(lr=learning_rate), 'Adagrad':Adagrad(lr=learning_rate), 'Adamax':Adamax(lr=learning_rate), 'Nadam':Nadam(lr=learning_rate), 'Ftrl':Ftrl(lr=learning_rate)}
    activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu', 'elu', 'exponential', 'LeakyReLU','relu']
    neurons = round(neurons)
    activation = activationL[round(activation)]
    batch_size = round(batch_size)
    epochs = round(epochs)
    def nn_cl_fun():
        opt = Adam(lr = learning_rate)
        nn = Sequential()
        nn.add(Dense(neurons, input_dim=178, activation=activation))
        nn.add(Dense(neurons, activation=activation))
        nn.add(Dense(1, activation='sigmoid'))
        nn.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
        return nn
    es = EarlyStopping(monitor='accuracy', mode='max', verbose=0, patience=20)
    nn = KerasClassifier(build_fn=nn_cl_fun, epochs=epochs, batch_size=batch_size, verbose=0)
    kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=123)
    score = cross_val_score(nn, X_train, Y_train, scoring=score_acc, cv=kfold, fit_params={'callbacks':[es]}).mean()
    return score

In [ ]:
params_nn ={ 'neurons': (10, 100), 'activation':(0, 9), 'optimizer':(0,7), 'learning_rate':(0.01, 1), 'batch_size':(200,
                                                                                                                    1000), 'epochs':(20, 100) }
# Run Bayesian Optimization
nn_bo = BayesianOptimization(nn_cl_bo, params_nn, random_state=111)
nn_bo.maximize(init_points=25, n_iter=4)

|   iter    |  target   | activa... | batch_... |  epochs   | learni... |  neurons  | optimizer |
-------------------------------------------------------------------------------------------------
|  1        |  0.7992   |  5.51     |  335.3    |  54.88    |  0.7716   |  36.58    |  1.044    |
|  2        |  0.7985   |  0.2023   |  536.2    |  39.09    |  0.3443   |  99.16    |  1.664    |
|  3        |  0.7985   |  0.7307   |  735.7    |  69.7     |  0.2815   |  51.96    |  0.8286   |
|  4        |  0.6791   |  0.6656   |  920.6    |  83.52    |  0.8422   |  83.37    |  6.937    |
|  5        |  0.8064   |  5.195    |  851.0    |  53.71    |  0.03717  |  50.87    |  0.7373   |
|  6        |  0.7985   |  7.355    |  758.2    |  65.22    |  0.2815   |  99.86    |  0.9663   |
|  7        |  0.7985   |  5.539    |  588.0    |  52.4     |  0.7306   |  39.05    |  2.804    |
|  8        |  0.7985   |  2.871    |  957.8    |  93.5     |  0.8157   |  13.07    |  6.604    |
|  9        |  0.798

In [ ]:
params_nn_ = nn_bo.max['params']
activationL = ['relu', 'sigmoid', 'softplus', 'softsign', 'tanh', 'selu',
               'elu', 'exponential', LeakyReLU,'relu']
params_nn_['activation'] = activationL[round(params_nn_['activation'])]

{'activation': 'relu',
 'batch_size': 200.0,
 'epochs': 100.0,
 'learning_rate': 0.01,
 'neurons': 100.0,
 'optimizer': 0.0}